In [ ]:
#hw3, question 2

In [1]:
Pkg.update()
Pkg.add("ForwardDiff")
Pkg.add
using ForwardDiff
using PyPlot

INFO: Updating METADATA...
INFO: Updating Toms566...
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Nothing to be done


LoadError: LoadError: ArgumentError: PyPlot not found in path
while loading In[1], in expression starting on line 5

In [2]:
#download("http://www.ats.ucla.edu/stat/data/binary.csv", "binary.csv")
A = convert(Array{Float64,2},readcsv("binary.csv")[2:end,:][:,1:3]);

In [3]:
function L(x,u)
    a = x[1:2]; β = x[3];
    uᵢ = u[2:3]; yᵢ = u[1]
    return -yᵢ*(vecdot(a,uᵢ) + β) + (1-yᵢ)log(1 + exp(vecdot(a,uᵢ) + β))
end;

In [4]:
G(x,ui) = ForwardDiff.gradient(x -> L(x,ui))(x)
H(x,ui) = ForwardDiff.hessian(x -> L(x,ui))(x)

m = size(A,1)
f(x) = sum([L(x, A[i,:]) for i in 1:m])
g(x) = sum([G(x, A[i,:]) for i in 1:m]);
h(x) = sum([H(x, A[i,:]) for i in 1:m])

h (generic function with 1 method)

In [5]:
function newtmin(obj, grd, hes, x0, n,maxIts=500, optTol=1e-5)
    # Minimize a function f using Newton’s method with modified Hessian
    
    # INPUT
    # obj: a function that evaluates the objective value, gradient and Hessian at point x, i.e. (f,g,H)=obj(x)
    # x0: starting point
    # maxIts (optional): maximum number of iterations.
    # optTol (optional): optimality tolerance based on: ||grad(x)|| <= optTol*||grad(x0)||
    
    # OUTPUT
    # xmin: Minimum point
    # fmin: Minimum 
    # iter: Number of iterations
    # fgrdnorm: 2-norm of gradient of test problem at minimum point
    
    # Using of Newton mathod, backtracking line search with Wolfe condition
    
    # initialize
    x = x0; # standard starting point
    
    alpha0 = 0.2; # para for backtracking line serch
    beta0 = 0.5;
    
    # loop with Newton Mehtod
    iter = 0; # number of iteration
    gradient_L_norm2 = 1;
    
    Opt = Float64[]
    
    while gradient_L_norm2 > optTol && iter < maxIts
        A = hes(x);
        
        # Non-negtive Hessian matrix check
        min_eigval_A = minimum(eigvals(A));
        if min_eigval_A < 0.0
            #println("NEGATIVE HESSIAN MATIRX!")
            #break
            beta = 1e-3;
            if minimum(diag(hes(x))) > 0
                tau = 0
            else
                tau = - minimum(diag(hes(x))) + beta;
            end
    
            while min_eigval_A < 0
                A = hes(x) + tau * eye(n)
                min_eigval_A = minimum(eigvals(A))
                tau = max(2 * tau, beta);
            end   
        end
        
        # compute Newton Step
        delta_x = - inv(A) * grd(x);
    
        # step size by line search
        t = 1.0;
        f_old = obj(x); # f_old := f(x)

        diff1 = 1.0; # difference of L/R Hand Side of Wolfe condition 
        diff2 = 1.0; # just initialize loop for backtracking
        
        c1 = 1/4; c2 = 1/2; # constent in Wolfe condition
        
        t = 1.0; # initial step
        while diff1[1] > 0 && diff2[1] > 0
            x_new = x + t * delta_x;
            f_new = obj(x_new); # f_new := f(x + \delta x)
        
            #wolfe condition
            diff1 = f_new - f_old - c1 * alpha0 * t * (grd(x))' * delta_x;
            diff2 = c2 * grd(x)' * delta_x - grd(x_new)' * delta_x;

            t = beta0 * t;
        end
 
        # update
        x = x + t * delta_x;
        
        #ratio = norm(grd(x)) / norm(grd(x0));
        gradient_L_norm2 = norm(grd(x));
        iter = iter + 1;
        
        # number of iteration check
        if iter >= maxIts
            println("REACH MAXIMUM ITERATION NUMBER!")
            break
        end
        
        push!(Opt, gradient_L_norm2)
        #println(gradient_L_norm2)
        
    end
    
    xmin = x;
    #fmin = obj(x);
    #fgrdnorm = norm(grd(x));
    return (xmin,Opt,iter);
    
end

newtmin (generic function with 3 methods)

In [6]:
(xmin,Opt,iter) = newtmin( x -> f(x), x -> g(x), x -> h(x), zeros(3) ,3, 500, 1e-6);

In [8]:
iter

27

In [7]:
plot(1:iter, log(Opt))

LoadError: LoadError: UndefVarError: plot not defined
while loading In[7], in expression starting on line 1